In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import argparse
import cv2

from PIL import Image, ImageOps
from numpy import asarray

import os
import numpy as np
import pandas as pd

In [3]:
# Jupyter only
from matplotlib import pyplot as plt
from IPython.display import display
%matplotlib inline

In [7]:
fps = '1FPS'
frame_dir = '../../data/DAiSEE/' + fps + '/data/' 
faces_dir = '../../data/DAiSEE/' + fps + '/dataFaces/'
caffe_dir = '../../data/models/caffe/'


In [9]:
# load model from disk
net = cv2.dnn.readNetFromCaffe(caffe_dir + 'deploy.prototxt.txt', caffe_dir + 'res10_300x300_ssd_iter_140000.caffemodel')
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)

In [10]:
# extract a single face from a given photograph
def extract_face(image_file, required_size=(300, 300)):
    image = cv2.imread(image_file)
    (h, w) = image.shape[:2]
    blob = cv2.dnn.blobFromImage(cv2.resize(image, (300,300)), 1.0, (300, 300), (103.93, 116.77, 123.68))
    net.setInput(blob)
    detections = net.forward()
    
    for i in range(0, detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.5:
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")
            face = image[startY:endY, startX:endX]
    return face

# load images and extract faces for all images in a directory
def load_faces(directory, destination):
    faces = list()
    cnt = 0
    for filename in os.listdir(directory):
        path = directory + filename
        if filename[-3:]=='jpg':
            try:
                face = extract_face(path)       
                cv2.imwrite(destination + filename, face)
                cnt +=1           
            except:
                continue
    
    return cnt

In [11]:
# Create image file structure for faces data and grab the faces
cols = ['b0', 'b1', 'b2', 'b3']
dirs = ['train', 'test', 'validation']

for d in dirs:
    for c in cols:
        in_dir = frame_dir + d + '/' + c + '/'
        out_dir = faces_dir + d + '/' + c + '/'
        
        # make the faces directory if it doesn't already exist
        if not os.path.exists(out_dir):
            os.makedirs(out_dir)
            
        # get the faces and save
        c = load_faces(in_dir, out_dir)
        print(out_dir, c)

../../data/DAiSEE/1FPS/dataFaces/train/b0/ 24325
../../data/DAiSEE/1FPS/dataFaces/train/b1/ 16955
../../data/DAiSEE/1FPS/dataFaces/train/b2/ 10726
../../data/DAiSEE/1FPS/dataFaces/train/b3/ 1560
../../data/DAiSEE/1FPS/dataFaces/test/b0/ 8230
../../data/DAiSEE/1FPS/dataFaces/test/b1/ 5837
../../data/DAiSEE/1FPS/dataFaces/test/b2/ 3373
../../data/DAiSEE/1FPS/dataFaces/test/b3/ 390
../../data/DAiSEE/1FPS/dataFaces/validation/b0/ 4459
../../data/DAiSEE/1FPS/dataFaces/validation/b1/ 3760
../../data/DAiSEE/1FPS/dataFaces/validation/b2/ 4750
../../data/DAiSEE/1FPS/dataFaces/validation/b3/ 1320
